# RAG Application with LangChain

## Introduction to RAG (Retrieval Augmented Generation)

**Retrieval Augmented Generation (RAG)** is a powerful technique that enhances Large Language Models (LLMs) by providing them with access to external data sources. This approach addresses a key limitation of LLMs: while they are trained on vast amounts of public data, they lack knowledge of your *private* data (e.g., company documents, personal files, or proprietary databases).

### How RAG Works

RAG operates through a three-step process:

1. **Retrieval**: Search and retrieve relevant information from your data source based on the user's query
2. **Augmentation**: Enhance the prompt sent to the LLM with the retrieved context
3. **Generation**: Use the LLM to generate a response grounded in the retrieved information

### What We'll Build

In this notebook, we will build a RAG application that can chat with PDF documents using **LangChain v1.0+**, the latest version of the popular framework for building LLM applications.

### Key Concepts Covered

| Concept | Description |
|---------|-------------|
| **Document Loading** | Reading files (PDFs, text, etc.) into a format LangChain can process |
| **Splitting** | Breaking documents into smaller, manageable chunks |
| **Embeddings** | Converting text into numerical vectors that capture semantic meaning |
| **Vector Store** | A database optimized for storing and searching these vectors |
| **Retrieval** | Finding the most relevant chunks for a given query |
| **Generation** | Using an LLM to synthesize an answer from retrieved context |

---

## Step 0: Install Required Packages

Before we begin, we need to install the required packages. This cell installs:

- **`langchain`**: The core LangChain framework
- **`langchain-community`**: Community integrations (document loaders, etc.)
- **`langchain-openai`**: OpenAI-specific components (embeddings, chat models)
- **`langchain-chroma`**: Chroma vector store integration
- **`langgraph`**: Graph-based agent orchestration (required for modern agents in v1.0+)
- **`pypdf`**: PDF parsing library
- **`gradio`**: Web interface for interactive demos
- **`python-dotenv`**: Environment variable management

> ⚠️ **Important**: After running this cell, you may need to **restart the kernel** to ensure all packages are properly loaded.

In [ ]:
# Install required packages
# Note: The --force-reinstall for numpy and scipy fixes potential binary incompatibility issues
# !uv pip install -U -q langchain langchain-community langchain-openai langchain-chroma langgraph pypdf gradio python-dotenv
# !uv pip install --force-reinstall -q numpy scipy

print("✅ Packages installed! Please restart the kernel if this is your first time running this cell.")

✅ Packages installed! Please restart the kernel if this is your first time running this cell.


---

## Step 1: Environment Setup

We need to configure our API keys to authenticate with OpenAI. This notebook supports both:

- **Google Colab**: Uses `google.colab.userdata` to securely access keys stored in Colab Secrets
- **Local Execution**: Uses `python-dotenv` to load keys from a `.env` file

### Setting Up Your API Key

**For local development**, create a `.env` file in this directory with:
```
OPENAI_API_KEY=your-api-key-here
```

**For Colab**, add your key to Colab Secrets with the name `OPENAI_API_KEY`.

In [ ]:
import os
import sys

# Configuration
MODEL = "gpt-4o-mini"  # The LLM model to use
db_name = "vector_db"  # Directory name for the vector store


# Option 1: Set your API key directly (for Colab)
#from google.colab import userdata
#os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Option 2:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Verify API key is set
if os.environ.get("OPENAI_API_KEY"):
    print("✅ OPENAI_API_KEY loaded successfully")
else:
    print("⚠️ Warning: OPENAI_API_KEY not found. Please set it in your .env file or environment.")

✅ OPENAI_API_KEY loaded successfully


---

## Step 2: Import Dependencies

Now we import all the necessary modules from LangChain and other libraries. Here's what each import does:

### Document Processing
- **`DirectoryLoader`**: Loads multiple files from a directory
- **`PyPDFLoader`**: Parses PDF files into text
- **`RecursiveCharacterTextSplitter`**: Splits text into chunks while respecting natural boundaries

### Embeddings & Vector Store
- **`OpenAIEmbeddings`**: Converts text to vector embeddings using OpenAI's models
- **`Chroma`**: A fast, open-source vector database

### LLM & Chains
- **`ChatOpenAI`**: OpenAI's chat models (GPT-4, etc.)
- **`create_history_aware_retriever`**: Creates a retriever that understands conversation context
- **`create_retrieval_chain`**: Combines retrieval and generation into a single chain
- **`create_stuff_documents_chain`**: Creates a chain that "stuffs" documents into the prompt

### Prompts & Messages
- **`ChatPromptTemplate`**: Templates for structured prompts
- **`MessagesPlaceholder`**: Placeholder for conversation history
- **`HumanMessage` / `AIMessage`**: Message types for chat history

In [3]:
import glob
import os

# Document loading and processing
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embeddings and LLM
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Vector store
from langchain_chroma import Chroma

# Chains for RAG
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# Prompts and messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# UI
import gradio as gr

print("✅ All imports successful!")

✅ All imports successful!


### 🔍 Detailed Breakdown of Imports

Let's break down exactly what each imported component does:

- **`DirectoryLoader, PyPDFLoader`**: `DirectoryLoader` helps us grab all files in a folder. `PyPDFLoader` is the specialist that knows how to read PDF files page by page.
- **`RecursiveCharacterTextSplitter`**: This is a "smart" splitter. Instead of just chopping text every 1000 characters (which might cut a sentence in half), it tries to split at natural boundaries like paragraphs `\n\n` or sentences `\n` to check context.
- **`OpenAIEmbeddings`**: This tool takes text and turns it into a list of numbers (vectors). It uses OpenAI's models to do this translation.
- **`Chroma`**: This is our database. It stores the vectors we created with `OpenAIEmbeddings` so we can search them later.
- **`create_history_aware_retriever`**: A special chain that takes your follow-up question (e.g., "How does it work?") and your chat history, and rewrites it into a full question (e.g., "How does RAG work?") so the database can understand it.
- **`create_retrieval_chain`**: The manager that coordinates everything: it gets the question, sends it to the retriever, gets documents back, and sends them to the LLM.
- **`create_stuff_documents_chain`**: The worker that actually sends the prompt to the LLM. It "stuffs" all the retrieved text into the system prompt.
- **`ChatPromptTemplate`**: A flexible template builder. It lets us create prompts with placeholders (like `{context}` or `{input}`) that get filled in dynamically.


---

## Step 3: Load Documents

The first step in building a RAG application is loading your documents. We use:

- **`glob.glob()`**: To find all PDF files in the `pdfs/` directory and current directory
- **`PyPDFLoader`**: To parse each PDF and extract text content

### Document Structure

Each loaded document contains:
- **`page_content`**: The actual text content
- **`metadata`**: Information about the document (source file, page number, etc.)

> 📁 **Note**: Place your PDF files in a `pdfs/` subdirectory or in the same directory as this notebook.

In [ ]:
# !pip install gdown -q

# !gdown 12hCcDOBYO0A3q2eFstMQnNqcCX2WYCZq -O pdfs.zip
# !unzip -q pdfs.zip -d pdfs
# !rm pdfs.zip
# print("✓ PDFs extracted to ./pdfs folder")

In [4]:
# Find all PDF files in the pdfs/ subdirectory and current directory
folders = glob.glob("pdfs/*.pdf") + glob.glob("*.pdf")

if not folders:
    print("⚠️ No PDF files found. Please add PDF files to the 'pdfs/' directory or current directory.")
else:
    print(f"📄 Found {len(folders)} PDF file(s)")

# Load all documents
documents = []
for file_path in folders:
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    for doc in docs:
        # Add custom metadata to track source file
        doc.metadata["source_file"] = os.path.basename(file_path)
        documents.append(doc)

print(f"✅ Loaded {len(documents)} pages from {len(folders)} file(s)")

📄 Found 9 PDF file(s)
✅ Loaded 126 pages from 9 file(s)


In [5]:
# print first document metadata, such as file name, source, total number of pages, etc.
print("First document metadata:")
print(documents[0].metadata['source'])
print(documents[0].metadata['total_pages'])
print(documents[0].metadata['source_file'])


First document metadata:
pdfs/1901.09069v2.pdf
11
1901.09069v2.pdf


In [6]:
# show the content of first document
print(documents[0].page_content)


Word Embeddings: A Survey
Felipe Almeida Geraldo Xex ´eo∗
Computer and Systems Engineering Program (PESC-COPPE)
Federal University of Rio de Janeiro
Rio de Janeiro, Brazil
{falmeida,xexeo}@cos.ufrj.br
Abstract
This work lists and describes the main re-
cent strategies for building ﬁxed-length,
dense and distributed representations for
words, based on the distributional hypoth-
esis. These representations are now com-
monly called word embeddings and, in ad-
dition to encoding surprisingly good syn-
tactic and semantic information, have been
proven useful as extra features in many
downstream NLP tasks.
1 Introduction
The task of representing words and documents is
part and parcel of most, if not all, Natural Lan-
guage Processing (NLP) tasks. In general, it has
been found to be useful to represent them as vec-
tors, which have an appealing, intuitive interpreta-
tion, can be the subject of useful operations (e.g.
addition, subtraction, distance measures, etc) and
lend themselves well to

In [7]:
print(len(documents[0].page_content))

3856


---

## Step 4: Split Documents into Chunks

LLMs have a **context window limit** (maximum tokens they can process at once). Additionally, for effective retrieval, we want to find *specific* relevant passages, not entire documents.

We use **`RecursiveCharacterTextSplitter`** which:
- Splits text hierarchically (paragraphs → sentences → words)
- Tries to keep semantically related text together
- Creates overlapping chunks to preserve context at boundaries

### Key Parameters

| Parameter | Value | Description |
|-----------|-------|-------------|
| `chunk_size` | 1000 | Maximum characters per chunk |
| `chunk_overlap` | 200 | Characters shared between adjacent chunks |
| `add_start_index` | True | Tracks the position of each chunk in the original document |

In [8]:
# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       # Max characters per chunk
    chunk_overlap=200,     # Overlap between chunks for context continuity
    add_start_index=True   # Track position in original document
)

# Split documents into chunks
chunks = text_splitter.split_documents(documents)
print(f"✅ Split {len(documents)} pages into {len(chunks)} chunks")

# Show example chunk
if chunks:
    print("\n📝 Example Chunk:")
    print("-" * 50)
    print(chunks[0].page_content[:300] + "...")
    print("-" * 50)
    print(f"Metadata: {chunks[0].metadata}")

✅ Split 126 pages into 688 chunks

📝 Example Chunk:
--------------------------------------------------
Word Embeddings: A Survey
Felipe Almeida Geraldo Xex ´eo∗
Computer and Systems Engineering Program (PESC-COPPE)
Federal University of Rio de Janeiro
Rio de Janeiro, Brazil
{falmeida,xexeo}@cos.ufrj.br
Abstract
This work lists and describes the main re-
cent strategies for building ﬁxed-length,
dense...
--------------------------------------------------
Metadata: {'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-05-03T00:58:57+00:00', 'author': '', 'keywords': '', 'moddate': '2023-05-03T00:58:57+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs/1901.09069v2.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1', 'source_file': '1901.09069v2.pdf', 'start_index': 0}


In [9]:
# inspect a chunk 
print(chunks[0].page_content)


Word Embeddings: A Survey
Felipe Almeida Geraldo Xex ´eo∗
Computer and Systems Engineering Program (PESC-COPPE)
Federal University of Rio de Janeiro
Rio de Janeiro, Brazil
{falmeida,xexeo}@cos.ufrj.br
Abstract
This work lists and describes the main re-
cent strategies for building ﬁxed-length,
dense and distributed representations for
words, based on the distributional hypoth-
esis. These representations are now com-
monly called word embeddings and, in ad-
dition to encoding surprisingly good syn-
tactic and semantic information, have been
proven useful as extra features in many
downstream NLP tasks.
1 Introduction
The task of representing words and documents is
part and parcel of most, if not all, Natural Lan-
guage Processing (NLP) tasks. In general, it has
been found to be useful to represent them as vec-
tors, which have an appealing, intuitive interpreta-
tion, can be the subject of useful operations (e.g.
addition, subtraction, distance measures, etc) and


In [10]:
print(len(chunks[0].page_content))

976


---

## Step 5: Create Embeddings and Vector Store

### What are Embeddings?

**Embeddings** are numerical representations (vectors) of text that capture semantic meaning. Texts with similar meanings will have vectors that are close together in the embedding space.

### What is a Vector Store?

A **Vector Store** is a specialized database optimized for:
- Storing high-dimensional vectors
- Performing fast similarity searches
- Enabling "semantic search" (finding text by meaning, not just keywords)

### Our Setup

- **`OpenAIEmbeddings`**: Uses OpenAI's `text-embedding-3-small` model (fast and cost-effective)
- **`Chroma`**: Open-source vector database that persists to disk

> 💡 **Tip**: The embeddings are stored locally, so subsequent runs will be faster as you won't need to re-embed documents.

In [11]:
# Initialize embedding model
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# Clean up existing database if it exists (to ensure fresh data)
if os.path.exists(db_name):
    import shutil
    shutil.rmtree(db_name)
    print(f"🗑️ Removed existing vector store: {db_name}")

# Create vector store from document chunks
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=db_name
)

print(f"✅ Vector store created with {vectorstore._collection.count()} documents")

✅ Vector store created with 688 documents


In [12]:
# test chroma db for similarity search 
query = "What is Byte Pair Encoding?"
docs_with_scores = vectorstore.similarity_search_with_score(query)
for doc, score in docs_with_scores:
    print(f"Document: {doc.page_content}\nScore: {score}\n")


Document: arXiv:2411.08671v1  [cs.DS]  13 Nov 2024
Theoretical Analysis of Byte-Pair Encoding
L´ aszl´ o Kozma and Johannes Voderholzer
Institut f¨ ur Informatik, Freie Universit¨ at Berlin, Germany
Abstract
Byte-Pair Encoding (BPE) is a widely used method for subword token ization, with origins in
grammar-based text compression. It is employed in a variety of lang uage processing tasks such
as machine translation or large language model (LLM) pretraining, t o create a token dictionary
of a prescribed size. Most evaluations of BPE to date are empirical, a nd the reasons for its good
practical performance are not well understood.
In this paper we focus on the optimization problem underlying BPE: ﬁn ding a pair encoding
that achieves optimal compression utility. We show that this problem is APX-complete, indi-
cating that it is unlikely to admit a polynomial-time approximation scheme . This answers, in a
stronger form, a question recently raised by Zouhar et al. [ ZMG+23].
Score: 0.63874

---

## Step 6: Build the RAG Chain

Now we create the complete RAG pipeline using **LangChain Expression Language (LCEL)**. The chain consists of two main components:

### 1. History-Aware Retriever

This component reformulates the user's question to be **standalone** (understandable without context). 

**Example:**
- Chat history: "Tell me about SecLM"
- Follow-up: "What are its main features?"
- Reformulated: "What are the main features of SecLM?"

### 2. Question-Answer Chain

This component:
1. Takes the retrieved documents and the question
2. "Stuffs" the documents into the prompt as context
3. Generates a grounded answer using the LLM

### The Complete Flow

```
User Question → Contextualize → Retrieve → Generate Answer
      ↑              ↓            ↓            ↓
  Chat History    Standalone    Relevant    Final
                   Question     Documents   Response
```

In [13]:
# 1. Initialize the LLM
llm = ChatOpenAI(temperature=0, model_name=MODEL)

In [14]:
# 2. Create a retriever from the vector store
# k=5 means we retrieve the top 5 most relevant chunks
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [18]:
query = "What is Byte Pair Encoding?"

# Just check what the retriever returns (raw documents)
docs = retriever.invoke(query)
print(f"Retrieved {len(docs)} documents")
len(docs)


Retrieved 5 documents


5

In [27]:
print(docs[0].page_content)

arXiv:2411.08671v1  [cs.DS]  13 Nov 2024
Theoretical Analysis of Byte-Pair Encoding
L´ aszl´ o Kozma and Johannes Voderholzer
Institut f¨ ur Informatik, Freie Universit¨ at Berlin, Germany
Abstract
Byte-Pair Encoding (BPE) is a widely used method for subword token ization, with origins in
grammar-based text compression. It is employed in a variety of lang uage processing tasks such
as machine translation or large language model (LLM) pretraining, t o create a token dictionary
of a prescribed size. Most evaluations of BPE to date are empirical, a nd the reasons for its good
practical performance are not well understood.
In this paper we focus on the optimization problem underlying BPE: ﬁn ding a pair encoding
that achieves optimal compression utility. We show that this problem is APX-complete, indi-
cating that it is unlikely to admit a polynomial-time approximation scheme . This answers, in a
stronger form, a question recently raised by Zouhar et al. [ ZMG+23].


In [60]:
for i, doc in enumerate(docs):
    print(f'---'*20)
    print(f'=== Document {i+1} sourced from {doc.metadata["source_file"]} page {doc.metadata["page"]} ===')
    print(f'=== Content of Document {i+1} ===')
    print(doc.page_content)
    

------------------------------------------------------------
=== Document 1 sourced from 2411.08671v1.pdf page 0 ===
=== Content of Document 1 ===
arXiv:2411.08671v1  [cs.DS]  13 Nov 2024
Theoretical Analysis of Byte-Pair Encoding
L´ aszl´ o Kozma and Johannes Voderholzer
Institut f¨ ur Informatik, Freie Universit¨ at Berlin, Germany
Abstract
Byte-Pair Encoding (BPE) is a widely used method for subword token ization, with origins in
grammar-based text compression. It is employed in a variety of lang uage processing tasks such
as machine translation or large language model (LLM) pretraining, t o create a token dictionary
of a prescribed size. Most evaluations of BPE to date are empirical, a nd the reasons for its good
practical performance are not well understood.
In this paper we focus on the optimization problem underlying BPE: ﬁn ding a pair encoding
that achieves optimal compression utility. We show that this problem is APX-complete, indi-
cating that it is unlikely to admit a polyn

In [61]:
query = "What is the capital of France?"
# This query might not be in the documents, so retrieval might return irrelevant info
docs = retriever.invoke(query)
for i, doc in enumerate(docs):
    print(f'---'*20)
    print(f'=== Document {i+1} sourced from {doc.metadata["source_file"]} page {doc.metadata["page"]} ===')
    print(f'=== Content of Document {i+1} ===')
    print(doc.page_content)

------------------------------------------------------------
=== Document 1 sourced from 1301.3781v3.pdf page 4 ===
=== Content of Document 1 ===
resulting vectors can be used to answer very subtle semantic relationships between words, such as
a city and the country it belongs to, e.g. France is to Paris as Germany is to Berlin. Word vectors
with such semantic relationships could be used to improve many existing NLP applications, such
as machine translation, information retrieval and question answering systems, and may enable other
future applications yet to be invented.
5
------------------------------------------------------------
=== Document 2 sourced from 1301.3781v3.pdf page 4 ===
=== Content of Document 2 ===
Somewhat surprisingly, these questions can be answered by performing simple algebraic operations
with the vector representation of words. To ﬁnd a word that is similar to small in the same sense as
biggest is similar to big, we can simply compute vectorX = vector(”biggest”)

In [65]:
from langchain_openai import ChatOpenAI

llm_basic = ChatOpenAI(model_name=MODEL, temperature=0)
# 3. Define the contextualization prompt
# This prompt helps reformulate questions based on chat history
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# Create the history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)



__What contextualize_q_system_prompt does__
* This prompt is used only by create_history_aware_retriever.​
* It tells the LLM: “Given chat history + latest user input, rewrite the question so it’s standalone; don’t answer it.” That rewritten question is then sent to the retriever.​
* You need this only if:
    * You want follow‑up questions like “What about its limitations?” to still retrieve the right chunks, and
    * You are using create_history_aware_retriever (or an equivalent “conversational retriever”).

If you don’t care about multi‑turn context in retrieval, you can skip the history‑aware retriever entirely and just use retriever = vectorstore.as_retriever(...) as you did with RetrievalQA. In that case, contextualize_q_system_prompt and its prompt are not needed.

In [66]:
# 4. Define the QA prompt
# This prompt instructs the LLM how to use the retrieved context
qa_system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)



__What qa_system_prompt does__
* This is the system prompt used by the answer‑generation step (create_stuff_documents_chain).​
* It controls how the LLM:
    * Uses {context} (retrieved docs),
    * Handles “I don’t know” cases,
    * Constrains length and style of answers
    

In [67]:
# 5. Combine into the final RAG chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

print("✅ RAG chain created successfully!")

✅ RAG chain created successfully!


---

## Step 7: Test the RAG Chain

Let's test our RAG chain with a simple query. The chain will:

1. Take the user's question
2. Retrieve relevant document chunks from the vector store
3. Generate a response based on the retrieved context

The response object contains:
- **`answer`**: The generated response
- **`context`**: The retrieved document chunks used to generate the answer

In [68]:
# Initialize empty chat history
chat_history = []

# Ask a question
query = "What is the main topic of these documents?"
response = rag_chain.invoke({"input": query, "chat_history": chat_history})

print("❓ Question:", query)
print("\n💬 Answer:", response["answer"])

# Update chat history for follow-up questions
chat_history.extend([
    HumanMessage(content=query),
    AIMessage(content=response["answer"])
])

print("\n✅ Chat history updated. You can now ask follow-up questions!")

❓ Question: What is the main topic of these documents?

💬 Answer: The main topic of these documents is the development and evaluation of universal text embeddings, which are models designed to perform well across a variety of natural language processing tasks. They discuss the challenges of creating effective embeddings, the importance of diverse and high-quality datasets, and recent advancements in the field. Additionally, they mention benchmarks like the Massive Text Embedding Benchmark (MTEB) that assess the performance of these models across multiple languages and tasks.

✅ Chat history updated. You can now ask follow-up questions!


In [69]:
query = "repeat the answer but this time in bullet points please"
response = rag_chain.invoke({"input": query, "chat_history": chat_history})

In [70]:
print("❓ Question:", query)
print("\n💬 Answer:", response["answer"])

❓ Question: repeat the answer but this time in bullet points please

💬 Answer: - The main topic is the development and evaluation of universal text embeddings for natural language processing tasks.
- It discusses challenges in creating effective embeddings and the importance of diverse, high-quality datasets.
- Recent advancements in the field and benchmarks like the Massive Text Embedding Benchmark (MTEB) are highlighted, assessing model performance across multiple languages and tasks.


In [71]:
query = "What is byte pair encoding?"
response = rag_chain.invoke({"input": query, "chat_history": chat_history})
print("❓ Question:", query)
print("\n💬 Answer:", response["answer"])

❓ Question: What is byte pair encoding?

💬 Answer: Byte Pair Encoding (BPE) is a data compression technique that iteratively replaces the most frequent pair of bytes in a sequence with a single, unused byte. It is adapted for word segmentation by merging characters or character sequences instead of bytes, creating a token dictionary of a prescribed size. BPE is commonly used in natural language processing tasks, such as machine translation, to improve tokenization and compression utility.


---

## Step 8: Interactive Chat with Gradio

We'll create a user-friendly chat interface using **Gradio**. This allows for:

- **Continuous conversation**: The chat history is maintained across turns
- **Shareable link**: Gradio provides a public URL (if `share=True`)
- **Easy testing**: No code needed to interact with your RAG application

### How It Works

1. The `predict` function converts Gradio's chat format to LangChain's message format
2. It invokes the RAG chain with the message and history
3. The response is returned to the Gradio interface

In [73]:
def predict(message, history):
    """
    Process a user message and return a response from the RAG chain.
    
    Args:
        message: The user's current message
        history: List of message dictionaries from Gradio 4.x+ (role/content format)
    
    Returns:
        The AI's response string
    """
    # Convert Gradio history to LangChain message format
    # Gradio 4.x+ uses [{'role': 'user'/'assistant', 'content': '...'}] format
    history_langchain_format = []
    for msg in history:
        if msg['role'] == 'user':
            history_langchain_format.append(HumanMessage(content=msg['content']))
        elif msg['role'] == 'assistant':
            history_langchain_format.append(AIMessage(content=msg['content']))
    
    # Invoke the RAG chain
    response = rag_chain.invoke({
        "input": message, 
        "chat_history": history_langchain_format
    })
    
    return response["answer"]

# Create and launch the Gradio interface
gr.ChatInterface(
    predict,
    title="📚 RAG Chat with PDF",
    description="Ask questions about your PDF documents. The AI will use the document content to provide accurate answers.",
    examples=[
        "Summarize the documents",
        "What are the key findings?",
        "What is the main topic?"
    ],
).launch(share=True, debug=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://e327e5a06577a278ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e327e5a06577a278ad.gradio.live


---

# Part 2: Chat with SQL Database

In this section, we explore how to use LangChain to interact with a **SQL database** using natural language. This is another powerful application of LLMs: translating human questions into SQL queries.

## Modern SQL Agent with LangChain v1.0+

LangChain v1.0+ introduces a modernized agent architecture using **`create_agent`** from `langchain.agents`. Key features:

| Feature | Description |
|---------|-------------|
| **Message-based interface** | Consistent API across all agent types |
| **System prompts** | Fine-grained control over agent behavior |
| **Built on LangGraph** | Enables persistence, human-in-the-loop, and advanced orchestration |
| **ReAct pattern** | Iterative reasoning and acting until a solution is found |

## How SQL Agents Work

```
User Question → Agent Reasoning → SQL Generation → Execute Query → Format Answer
                     ↓                  ↓              ↓
              "What tables exist?"  "SELECT..."   "8 employees"
```

---

## Step 9: Set Up the SQL Agent

We'll use the **Chinook database**, a sample database representing a digital music store. It contains tables for:
- Artists, Albums, Tracks
- Customers, Employees, Invoices

### Components Used

- **`SQLDatabase`**: Connects to and inspects the database
- **`SQLDatabaseToolkit`**: Provides tools for the agent to interact with SQL
- **`create_agent`**: Creates a ReAct agent with the SQL tools

> ⚠️ **Security Note**: SQL agents execute model-generated queries. In production, use read-only database connections and implement proper query validation.

### `create_agent` + toolkit tools
* __Purpose__: Generic agent builder that works with any tools, not just SQL; it builds a graph-based agent runtime under the hood
* __Interface__: Message-centric. You pass `{"messages": [...]}` and it returns a messages list; you control the system prompt explicitly.
* __Flexibility__: Very configurable (middleware, custom graph nodes, more complex orchestration) and is the forward-looking v1 way to build agents.
* __SQL behavior__: You are manually giving it the SQL tools from `SQLDatabaseToolkit.get_tools()`, so the “SQL-ness” is just a set of generic tools plus your system prompt; there is no extra SQL-specific prompt engineering or guardrails baked in for you.​
* __Conceptually__: use this when you want a general agent framework that happens to have SQL tools, and you are comfortable owning the prompt/spec and any extra behaviors.

In [74]:
import requests
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_agent

# Download the Chinook sample database if it doesn't exist
db_url = "https://github.com/laxmimerit/All-CSV-ML-Data-Files-Download/raw/refs/heads/master/db_samples/Chinook.db"
if not os.path.exists("Chinook.db"):
    print("📥 Downloading Chinook database...")
    response = requests.get(db_url)
    if response.status_code == 200:
        with open("Chinook.db", "wb") as file:
            file.write(response.content)
        print("✅ Chinook.db downloaded successfully!")
    else:
        print(f"❌ Failed to download database (status: {response.status_code})")

# Connect to the database
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(f"\n📊 Available tables: {db.get_usable_table_names()}")


📊 Available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [75]:
result = db.run("SELECT * FROM ALBUM LIMIT 10")
print(result)

[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3), (6, 'Jagged Little Pill', 4), (7, 'Facelift', 5), (8, 'Warner 25 Anos', 6), (9, 'Plays Metallica By Four Cellos', 7), (10, 'Audioslave', 8)]


In [76]:
result = db.run("SELECT * FROM ARTIST LIMIT 10")
print(result)

[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]


In [77]:
# Create the SQL toolkit with tools for the agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Define the agent's system prompt
system_prompt = (
    "You are an agent designed to interact with a SQL database. "
    "Given an input question, create a syntactically correct SQLite query to run, "
    "then look at the results of the query and return the answer. "
    "You can order the results by a relevant column to return the most interesting examples. "
    "Never query for all the columns from a specific table, only ask for the relevant columns. "
    "You have access to tools for interacting with the database. "
    "Only use the information returned by these tools to construct your final answer. "
    "If you get an error while executing a query, rewrite the query and try again."
)

# Create the agent using LangChain v1.0+ API
agent = create_agent(
    model=llm,
    tools=toolkit.get_tools(),
    system_prompt=system_prompt
)

print("✅ SQL Agent created successfully!")

✅ SQL Agent created successfully!


---

## Step 10: Test the SQL Agent

Let's test our SQL agent with some natural language queries. The agent will:

1. Understand the question
2. Explore the database schema if needed
3. Generate and execute SQL queries
4. Interpret results and provide a human-readable answer

Notice how we use a **message-based interface** with the agent - this is the modern LangChain v1.0+ pattern.

In [78]:
# Test the SQL agent with a simple question
question = "How many employees are there?"

print(f"❓ Question: {question}\n")

# Invoke the agent with the message-based interface
response = agent.invoke(
    {"messages": [{"role": "user", "content": question}]}
)

# Extract the final answer from the response
print(f"💬 Answer: {response['messages'][-1].content}")

❓ Question: How many employees are there?

💬 Answer: There are 8 employees.


---

## Step 11: Try More Complex Queries

The SQL agent can handle more sophisticated questions that require:
- Joining multiple tables
- Aggregations and grouping
- Filtering and sorting

Try some of these example queries!

In [81]:
# Example complex queries to try
complex_queries = [
    "What are the top 5 best-selling artists by total sales?",
    "How many tracks are in each genre?",
    "Who are the customers from the USA?",
]

# Test with the first complex query
query = "What are the top 3 genres by number of tracks?"

print(f"❓ Question: {query}\n")

response = agent.invoke(
    {"messages": [{"role": "user", "content": query}]}
)

print(f"💬 Answer: {response['messages'][-1].content}")

❓ Question: What are the top 3 genres by number of tracks?

💬 Answer: The top 3 genres by number of tracks are:

1. **Rock** - 1297 tracks
2. **Latin** - 579 tracks
3. **Metal** - 374 tracks


### `create_sql_agent` AgentExecutor helper 
* __Purpose__: A convenience helper that specializes an agent for SQL: it wires tools, default prompts, and an agent loop tuned for querying databases.
* __SQL behavior__: It automatically builds the SQL toolkit (if you just pass db=) or uses your toolkit, attaches the right tools, and uses a prebuilt prompt for SQL Q&A (including instructions like only selecting relevant columns, etc.)
* __Status__: This is “LangChain Classic” style but still supported; it is simpler for pure SQL use cases and is what many SQL tutorials still use
* __Conceptually__: use this when you primarily want a database Q&A agent and prefer a one-call setup with built-in SQL prompts, easy verbosity, and AgentExecutor semantics

### 🛠️ Understanding the SQL Components

- **`SQLDatabase`**: Validates your connection string and inspects the database tables/schema automatically.
- **`SQLDatabaseToolkit`**: A bundle of "tools" that the agent can use. It gives the agent the ability to: `list_tables_sql_db`, `schema_sql_db`, and `query_sql_db`.
- **`create_sql_agent`**: A factory function that wires up the LLM, the toolkit, and a specific system prompt designed for SQL generation. It handles the "loop" of: Thought -> Action (Query) -> Observation (Result) -> Answer.


In [82]:
from langchain_community.agent_toolkits import create_sql_agent

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

result = agent.invoke({"input": "How many employees are there?"})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe "Employee" table seems to be the most relevant for finding the number of employees. I will check the schema of the "Employee" table to understand its structure.  
Action: sql_db_schema  
Action Input: "Employee"  
CREATE TABLE "Employee" (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Employee table:
EmployeeId	LastName	FirstName	Titl

In [83]:
result

{'input': 'How many employees are there?', 'output': 'There are 8 employees.'}

In [84]:
result = agent.invoke({"input": "What are the top 3 genres by number of tracks?"})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI need to check the schema of the "Track" and "Genre" tables to understand how to count the number of tracks per genre.  
Action: sql_db_schema  
Action Input: "Track, Genre"  
CREATE TABLE "Genre" (
	"GenreId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("GenreId")
)

/*
3 rows from Genre table:
GenreId	Name
1	Rock
2	Jazz
3	Metal
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("Alb

In [85]:
result

{'input': 'What are the top 3 genres by number of tracks?',
 'output': 'The top 3 genres by number of tracks are Rock (1297 tracks), Latin (579 tracks), and Metal (374 tracks).'}

---

# Summary

In this notebook, you learned how to build two powerful AI applications using **LangChain v1.0+**:

## Part 1: RAG Application
- 📄 **Document Loading**: Load and parse PDF files
- ✂️ **Text Splitting**: Break documents into semantic chunks
- 🔢 **Embeddings**: Convert text to vector representations
- 🗄️ **Vector Store**: Store and search vectors with Chroma
- 🔗 **RAG Chain**: Combine retrieval and generation
- 💬 **Chat Interface**: Build an interactive UI with Gradio

## Part 2: SQL Agent
- 🗃️ **Database Connection**: Connect to SQLite databases
- 🤖 **Agent Creation**: Build a ReAct agent with SQL tools
- 🔍 **Natural Language Queries**: Translate questions to SQL

## Key LangChain v1.0+ Features Used
- `create_agent` for building agents (replaces older patterns)
- Message-based interface for consistent API
- Built on LangGraph for advanced orchestration

## Next Steps
- Try adding more documents to your RAG application
- Experiment with different chunk sizes and overlap values
- Connect to your own SQL databases
- Explore LangChain's other agent types and tools